# Development Discussion for EmoPy

We will use this notebook to illustrate the scraping of the [Unicode Emoji List](https://unicode.org/emoji/charts/full-emoji-list.html). While there are likely many more uses for having an easily accessible Python repository of emojis, on creation the purpose was to create a set of tools to make searching and extracting emojis from common social media text easier. 

### Acquiring

In [1]:
import requests as rq
from lxml import html

In [2]:
# Get the emojis and their descriptions
resp = rq.get("https://unicode.org/emoji/charts/full-emoji-list.html")
html_tree = html.fromstring(resp.text)
emojis = html_tree.xpath('//td[@class="chars"]//text()')
descrs = html_tree.xpath('//td[contains(@class,"name")]//text()')

for idx, (e, d) in enumerate(zip(emojis, descrs)):
    print(f'Emoji: {e} || Descr: {d}')
    if idx >= 5:
        break

Emoji: 😀 || Descr: grinning face
Emoji: 😃 || Descr: grinning face with big eyes
Emoji: 😄 || Descr: grinning face with smiling eyes
Emoji: 😁 || Descr: beaming face with smiling eyes
Emoji: 😆 || Descr: grinning squinting face
Emoji: 😅 || Descr: grinning face with sweat


### Sorting

What we will quickly realize on exploration is that emojis can be several characters long, and often the longer (in terms of number of characters) emojis are often "glued" together versions of shorter emojis, often glued with the unicode `\u200d` character. Since we will want to search for the actual, intended emoji, we will sort these from longest to shortest. 

To be clear, imagine that we had "emojis" `A+B`, `A`, and `B`. Then, if we searched for `A` or `B`, we would find them; however, we really want to find `A+B` and not give the limelight to the shorter "building blocks" of the true emoji. This leads us to search by first looking for `A+B`; if that's not there, then search for `A`; if not there, then `B`. This is the perspective of ordering longer-to-shorter when we search.

In [3]:
emojis, descrs = zip(*sorted(list(zip(emojis, descrs)), key=lambda x: len(x[0]), reverse=True))

In [4]:
for idx, (e, d) in enumerate(zip(emojis, descrs)):
    print(f'Emoji: {e} || Length: {len(e)} || Descr: {d}')
    if idx >= 5:
        break

Emoji: 👩‍❤️‍💋‍👨 || Length: 8 || Descr: kiss: woman, man
Emoji: 👨‍❤️‍💋‍👨 || Length: 8 || Descr: kiss: man, man
Emoji: 👩‍❤️‍💋‍👩 || Length: 8 || Descr: kiss: woman, woman
Emoji: 👨‍👩‍👧‍👦 || Length: 7 || Descr: family: man, woman, girl, boy
Emoji: 👨‍👩‍👦‍👦 || Length: 7 || Descr: family: man, woman, boy, boy
Emoji: 👨‍👩‍👧‍👧 || Length: 7 || Descr: family: man, woman, girl, girl


### Searching via RegEx

We will create a regular expression of the form `(A+B|A|B)` so that if `A+B` appears, it will not match specifically `A` nor `B`, but will if they are separate instances. Since we already have ordered the emojis from longest character length to shortest, you would think that all that remains is to compile the regular expression 
```
emojis_re_str = "(" + "|".join(emojis)) + ")"
emojis_re = re.compile(emojis_re_str)
```
However, you will run into an error if you try this. The reason for the error is because of the asterisk character `*` which the interpreter wants as a regex repeat, but which takes place in the emoji `emojis[223]` (take a look!). So, we need to enter a break character:

In [5]:
import re

In [6]:
emojis_re_str = "(" + "|".join([el.replace("*","\*") for el in emojis]) + ")"
emojis_re = re.compile(emojis_re_str)

To show that this works as expected, consider the first emoji in our list `emojis[0]`. This emoji is of a man and woman kissing and is created by concatenating the emojis for a woman, the emoji of a heart, the emoji of kiss lips, and finally with the emoji of a man. Take a look, and notice that the "glue" is the unicode `\u200d`:

In [7]:
list(emojis[0])

['👩', '\u200d', '❤', '️', '\u200d', '💋', '\u200d', '👨']

Now, we will create a false sentence that one might see in Twitter which contains both the 8-character kiss emoji, a woman, a man, and a heart emoji:

In [8]:
em_breakdown = list(emojis[0])
s = f'Hi!! :) I love you {emojis[0]}{em_breakdown[0]}{em_breakdown[-1]}{em_breakdown[2]}'
print(s)

Hi!! :) I love you 👩‍❤️‍💋‍👨👩👨❤


Let's extract the emojis that we find with this regex:

In [9]:
for e in re.findall(emojis_re,s):
    print(e)

👩‍❤️‍💋‍👨
👩
👨
❤


Bingo! Notice that we did not duplicate any emojis that appeared in the "glued" expression, nor forget the ones which were singularly apart from that expression.